<a href="https://colab.research.google.com/github/george-kan/fastai/blob/main/Mnist_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook

     |████████████████████████████████| 727kB 5.9MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 1.2MB 31.9MB/s 
     |████████████████████████████████| 194kB 27.9MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
(path/'training').ls()

(#10) [Path('/root/.fastai/data/mnist_png/training/8'),Path('/root/.fastai/data/mnist_png/training/3'),Path('/root/.fastai/data/mnist_png/training/4'),Path('/root/.fastai/data/mnist_png/training/0'),Path('/root/.fastai/data/mnist_png/training/6'),Path('/root/.fastai/data/mnist_png/training/9'),Path('/root/.fastai/data/mnist_png/training/5'),Path('/root/.fastai/data/mnist_png/training/7'),Path('/root/.fastai/data/mnist_png/training/1'),Path('/root/.fastai/data/mnist_png/training/2')]

In [5]:
folder = 'training/3'

(path/folder).ls()

(#6131) [Path('/root/.fastai/data/mnist_png/training/3/45599.png'),Path('/root/.fastai/data/mnist_png/training/3/46500.png'),Path('/root/.fastai/data/mnist_png/training/3/41404.png'),Path('/root/.fastai/data/mnist_png/training/3/10993.png'),Path('/root/.fastai/data/mnist_png/training/3/983.png'),Path('/root/.fastai/data/mnist_png/training/3/27840.png'),Path('/root/.fastai/data/mnist_png/training/3/17333.png'),Path('/root/.fastai/data/mnist_png/training/3/29097.png'),Path('/root/.fastai/data/mnist_png/training/3/40205.png'),Path('/root/.fastai/data/mnist_png/training/3/39761.png')...]

In [6]:
(path/'testing').ls()

(#10) [Path('/root/.fastai/data/mnist_png/testing/8'),Path('/root/.fastai/data/mnist_png/testing/3'),Path('/root/.fastai/data/mnist_png/testing/4'),Path('/root/.fastai/data/mnist_png/testing/0'),Path('/root/.fastai/data/mnist_png/testing/6'),Path('/root/.fastai/data/mnist_png/testing/9'),Path('/root/.fastai/data/mnist_png/testing/5'),Path('/root/.fastai/data/mnist_png/testing/7'),Path('/root/.fastai/data/mnist_png/testing/1'),Path('/root/.fastai/data/mnist_png/testing/2')]

In [7]:
train_x = []
train_y = []
test_x = []
test_y = []
for digit in range(10):
    folder = 'training/' + str(digit)
    new_inp = [tensor(Image.open(x)) for x in (path/folder).ls()]
    print(digit, len(new_inp))
    train_x += new_inp
    train_y += [digit]*len(new_inp)
    test_folder = 'testing/' + str(digit)
    test_inp = [tensor(Image.open(x)) for x in (path/test_folder).ls()]
    test_x += test_inp
    test_y += [digit]*len(test_inp)
    

0 5923
1 6742
2 5958
3 6131
4 5842
5 5421
6 5918
7 6265
8 5851
9 5949


In [8]:
train_x_stacked = torch.stack(train_x).view(-1, 28*28).float()/255
train_y_stacked = tensor(train_y).view(-1, 1)
train_ds = list(zip(train_x_stacked, train_y_stacked))
train_dl = DataLoader(train_ds, batch_size= 256, shuffle=True)

test_x_stacked = torch.stack(test_x).view(-1, 28*28).float()/255
test_y_stacked = tensor(test_y).view(-1, 1)
test_ds = list(zip(test_x_stacked, test_y_stacked))
test_dl = DataLoader(test_ds, batch_size=256)

dls = DataLoaders(train_dl, test_dl)

train_x_stacked.shape, train_y_stacked.shape, test_x_stacked.shape, test_y_stacked.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [9]:
train_x[2000]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  12,  92, 214, 255, 254, 176, 138,  33,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 143, 253, 253, 253, 253, 253, 253, 2

In [10]:
df = pd.DataFrame(torch.trunc(tensor(train_x[2000]).float()))
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,92.000000,214.000000,255.000000,254.000000,176.000000,138.000000,33.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,143.000000,253.000000,253.000000,253.000000,253.000000,253.000000,253.000000,236.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,88.000000,250.000000,221.000000,93.000000,10.000000,27.000000,192.000000,251.000000,253.000000,240.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54.000000,246.000000,199.000000,28.000000,0.000000,0.000000,0.000000,0.000000,76.000000,230.000000,253.000000,208.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,150.000000,215.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,251.000000,253.000000,142.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
def batch_accuracy(x, y):
    preds = torch.argmax(x, 1)
    correct = preds == y
    return correct.float().mean()

simple_net = nn.Sequential(
            nn.Linear(28*28, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 10)
)
loss = CrossEntropyLossFlat()
learn = Learner(dls, simple_net, opt_func = SGD, loss_func = loss, metrics = batch_accuracy)
learn.fit(10, 0.3)


epoch,train_loss,valid_loss,batch_accuracy,time
0,0.342673,0.273259,0.844673,00:01
1,0.211816,0.181899,0.866217,00:01
2,0.167797,0.272501,0.841799,00:01
3,0.144669,0.131778,0.883945,00:01
4,0.122090,0.131966,0.882782,00:01
5,0.103729,0.173585,0.869960,00:01
6,0.090275,0.111314,0.887486,00:01
7,0.082080,0.118197,0.885019,00:01
8,0.081032,0.115382,0.887778,00:01
9,0.074782,0.095189,0.893688,00:01


In [ ]:
x,y = dls.one_batch()
learn.get_preds(dl = [(x,y)])

In [13]:
??Learner

In [69]:
def init_params(size1, size2): return torch.randn(size1, size2).requires_grad_()

def my_linear1(x): return x@w1 + b1

def my_linear2(x): return x@w2

#def my_relu(x): return torch.maximum(tensor(0), x)
def my_relu(x): return torch.where(x < 0, 0.01*x, x)

def my_softmax(x): 
    # Normalize the input to avoid numerical problems
    x = x - torch.max(x, dim = 1).values.view(-1, 1)
    return torch.exp(x) / torch.exp(x).sum(dim = 1).unsqueeze(1)

def my_loss(x, y): 
    x = my_softmax(x)
    y = y.view(-1, 1)
    selection = torch.gather(x, 1, y)
    #print(selection)
    #return (1-selection).mean()
    return (-torch.log(selection + 1e-15)).mean()

def my_model(x):
    lay1 = my_linear1(x)
    nonlin = my_relu(lay1)
    #return nonlin
    preds = my_linear2(nonlin)
    return(preds)

def update_grads(preds, y, params):
    loss = my_loss(preds, y)
    #print(loss)
    loss.backward()
    for p in params:
        my_grad = p.grad
        p.data -= my_grad*lr
        #print(p.mean())
        #print(my_grad.mean())
        p.grad.zero_()

def eval_batch():
    preds = my_model(test_x_stacked)
    positions = torch.max(preds, dim = 1).indices
    return (positions == test_y_stacked).float().mean()

w1 = init_params(28*28, 40)
#w1 = init_params(28*28, 10)
b1 = init_params(1, 1)
w2 = init_params(40, 10)
#b2 = init_params(1, 1)
lr = 1
params = w1, b1, w2#, b2

In [26]:
my_loss(tensor([[1,2,3], [2,3,4]]).float(), tensor([[0],[2]]))

tensor(0.6224)

In [27]:
my_softmax(tensor([[1,2,3], [2,3,4]]).float())

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])

In [17]:
-torch.log(tensor([0.0900, 0.6652])).mean()

tensor(1.4078)

In [34]:
inputs, classes = next(iter(train_dl))  

In [67]:
for i in range(40):
   preds = my_model(inputs)
   loss = my_loss(preds, classes)
   print(loss)
   update_grads(preds, classes, params)


tensor(28.3323, grad_fn=<MeanBackward0>)
tensor(26.7974, grad_fn=<MeanBackward0>)
tensor(25.7820, grad_fn=<MeanBackward0>)
tensor(25.9959, grad_fn=<MeanBackward0>)
tensor(27.1626, grad_fn=<MeanBackward0>)
tensor(26.3554, grad_fn=<MeanBackward0>)
tensor(24.6923, grad_fn=<MeanBackward0>)
tensor(20.2098, grad_fn=<MeanBackward0>)
tensor(15.6669, grad_fn=<MeanBackward0>)
tensor(13.9213, grad_fn=<MeanBackward0>)
tensor(11.3736, grad_fn=<MeanBackward0>)
tensor(9.8695, grad_fn=<MeanBackward0>)
tensor(6.8401, grad_fn=<MeanBackward0>)
tensor(5.5537, grad_fn=<MeanBackward0>)
tensor(3.6629, grad_fn=<MeanBackward0>)
tensor(2.0663, grad_fn=<MeanBackward0>)
tensor(1.5061, grad_fn=<MeanBackward0>)
tensor(1.2557, grad_fn=<MeanBackward0>)
tensor(0.9283, grad_fn=<MeanBackward0>)
tensor(0.7567, grad_fn=<MeanBackward0>)
tensor(0.5841, grad_fn=<MeanBackward0>)
tensor(0.4421, grad_fn=<MeanBackward0>)
tensor(0.3738, grad_fn=<MeanBackward0>)
tensor(0.3359, grad_fn=<MeanBackward0>)
tensor(0.3075, grad_fn=<MeanB

In [19]:
preds = my_model(inputs)


In [20]:
my_loss(preds, classes)

tensor(inf, grad_fn=<MeanBackward0>)

In [21]:
preds

tensor([[  48.5758,   41.9874,  -23.8285,  ...,  -19.1772,  -58.3082,    9.6861],
        [  15.4339,   54.5281,  -23.6561,  ...,    9.9141,  -50.3700,   17.9040],
        [  47.5532,  -10.2393,    9.2663,  ...,  -27.8686,  -19.2552,   20.2736],
        ...,
        [   4.9536,   32.6464, -105.7664,  ...,  -32.9201,    4.0790,   21.7409],
        [  -6.2516,    8.2867,  -16.7546,  ...,    3.3335,    6.3182,   31.1750],
        [  54.9024,   19.8724,  -22.8912,  ...,    1.2449,  -12.9686,   40.9418]], grad_fn=<MmBackward>)

In [22]:
my_softmax(preds)

tensor([[2.0548e-16, 2.8280e-19, 0.0000e+00,  ..., 1.4013e-45, 0.0000e+00, 2.6497e-33],
        [6.3184e-25, 6.0118e-08, 6.6688e-42,  ..., 2.5317e-27, 0.0000e+00, 7.4710e-24],
        [5.4000e-19, 4.3440e-44, 1.2723e-35,  ..., 0.0000e+00, 0.0000e+00, 7.6740e-31],
        ...,
        [3.5608e-34, 3.7875e-22, 0.0000e+00,  ..., 0.0000e+00, 1.4849e-34, 6.9532e-27],
        [8.0906e-29, 1.6667e-22, 2.2210e-33,  ..., 1.1769e-24, 2.3279e-23, 1.4526e-12],
        [1.8263e-11, 1.1174e-26, 2.8026e-45,  ..., 9.0872e-35, 6.1034e-41, 1.5796e-17]], grad_fn=<DivBackward0>)

In [23]:
y = y.view(-1, 1)
    selection = torch.gather(x, 1, y)

IndentationError: ignored

In [80]:
for i in range(40):
  losses = 0
  for x,y in train_dl:
      #print(torch.unique(y, return_counts = True))
      preds = my_model(x)
      losses += my_loss(preds, y)
      train_len = len(y)
      update_grads(preds, y, params)
      #print('break')
  #print(losses/train_len)
  val_preds = my_model(test_x_stacked)
  print(my_loss(val_preds, test_y_stacked))
  

tensor(0.2612, grad_fn=<MeanBackward0>)
tensor(0.2468, grad_fn=<MeanBackward0>)
tensor(0.2442, grad_fn=<MeanBackward0>)
tensor(0.2592, grad_fn=<MeanBackward0>)
tensor(0.2764, grad_fn=<MeanBackward0>)
tensor(0.2562, grad_fn=<MeanBackward0>)
tensor(0.2476, grad_fn=<MeanBackward0>)
tensor(0.2518, grad_fn=<MeanBackward0>)
tensor(0.2496, grad_fn=<MeanBackward0>)
tensor(0.2418, grad_fn=<MeanBackward0>)
tensor(0.2450, grad_fn=<MeanBackward0>)
tensor(0.2785, grad_fn=<MeanBackward0>)
tensor(0.2410, grad_fn=<MeanBackward0>)
tensor(0.3688, grad_fn=<MeanBackward0>)
tensor(0.2425, grad_fn=<MeanBackward0>)
tensor(0.2437, grad_fn=<MeanBackward0>)
tensor(0.2448, grad_fn=<MeanBackward0>)
tensor(0.2411, grad_fn=<MeanBackward0>)
tensor(0.2559, grad_fn=<MeanBackward0>)
tensor(0.2410, grad_fn=<MeanBackward0>)
tensor(0.2398, grad_fn=<MeanBackward0>)
tensor(0.2431, grad_fn=<MeanBackward0>)
tensor(0.2424, grad_fn=<MeanBackward0>)
tensor(0.2445, grad_fn=<MeanBackward0>)
tensor(0.2372, grad_fn=<MeanBackward0>)


RuntimeError: ignored